In [1]:
from typing import Literal

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_ollama import ChatOllama

/Users/minghuang/miniconda3/envs/llm/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3579: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


In [2]:
# Data model
class RouteQuery(BaseModel):
    """Route a user query to the most relevant datasource."""

    # the type must be a literal string that is either "python_docs" or "js_docs"
    datasource: Literal["Python", "JavaScript"] = Field(
        ..., # Required, means that the field must be present in the input
        description="""Given a user question, choose which datasource would be most relevant for answering their question""",
    )

In [3]:
# LLM with function call 
llm = ChatOllama(base_url="http://localhost:11434", model="qwen2.5:32b", temperature=0)
structured_llm = llm.with_structured_output(RouteQuery)

# Prompt 
system = """You are an expert at routing a user question to the appropriate data source.

Based on the programming language the question is referring to, route it to the relevant data source."""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{question}"),
    ]
)

# Define router 
router = prompt | structured_llm

In [4]:
question = """Why doesn't the following code work:

from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(["human", "speak in {language}"])
prompt.invoke("french")
"""

In [5]:
result = router.invoke({"question": question})

In [6]:
result

RouteQuery(datasource='Python')